# Gradient Boosting - Performance with Discipline (and Leakage Avoidance)

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/13_gradient_boosting_student.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Explain boosting vs bagging at a high level
2. Train a gradient boosting model with sensible defaults
3. Tune learning rate / depth / estimators with runtime controls
4. Compare boosted model vs forest under consistent CV
5. Identify and control overfitting in boosting

---

> **📋 Participation Reminder:** This notebook contains **2 PAUSE-AND-DO exercises**. You are expected to complete all exercises before submitting your notebook.

---

## 💼 Why This Matters: Learning from Mistakes, One Tree at a Time

The **State Health Department** wants to build a second-opinion system — an automated tool that flags cases where the primary model and a boosting model disagree, triggering a senior pathologist review. Unlike random forests (which build trees independently), gradient boosting builds them sequentially: each new tree focuses specifically on the cases the previous trees got wrong.

This "learn from your mistakes" strategy often produces the most accurate models in practice. Companies like Google, Netflix, and major healthcare systems rely on gradient boosting for their most demanding prediction tasks.

> **Today's focus:** Understanding the boosting paradigm, building GradientBoosting and HistGradientBoosting classifiers, and comparing them against random forests on the breast cancer dataset.

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import randint, uniform
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

**Reading the output:**

The setup cell imports `GradientBoostingClassifier` from scikit-learn's ensemble module and `RandomizedSearchCV` for efficient hyperparameter tuning. The `scipy.stats` distributions (`randint`, `uniform`) define the random search spaces we will use later.

The confirmation message `Setup complete!` with **RANDOM_SEED = 474** means all stochastic operations (data splits, random search iterations, bootstrap samples inside the booster) will produce identical results every time you run this notebook.

**Key takeaway:** Gradient Boosting is deterministic given a fixed seed, unlike some implementations of random forests that use parallel randomness. This makes debugging easier but also means a single seed can give atypically good or bad results, which is why we always use cross-validation.

---

## 1. Boosting vs Bagging

### Bagging (Random Forest)
- **Strategy**: Train many trees independently on bootstrap samples
- **Combine**: Average (or vote) predictions
- **Effect**: Reduces variance
- **Trees**: Can be deep (low bias, high variance)
- **Parallelizable**: Yes

### Boosting (Gradient Boosting)
- **Strategy**: Train trees sequentially, each correcting previous errors
- **Combine**: Weighted sum of predictions
- **Effect**: Reduces bias (and some variance)
- **Trees**: Should be shallow (reduce variance of individual trees)
- **Parallelizable**: Not really (sequential by nature)

### Gradient Boosting Algorithm (Simplified)

```
1. Start with initial prediction (e.g., mean)
2. For iteration m = 1 to M:
   a. Calculate residuals (errors) from current model
   b. Fit shallow tree to predict residuals
   c. Add tree to model with learning rate η:
      F_m(x) = F_(m-1)(x) + η * tree_m(x)
3. Final model: F_M(x) = sum of all trees
```

**Key insight**: Each tree tries to fix mistakes of previous trees

> 💡 **Gemini Prompt:** "Load Breast Cancer dataset, split 70/30 with stratification (seed 474), print sizes."
>
> **After running, verify:**
> - Train ~398 samples, test ~171
> - Target is binary (0/1)
> - Split uses stratify=y


In [ ]:
# Load data
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y)

print(f"Train: {len(X_train)} | Test: {len(X_test)}")

**Reading the output:**

The data is split into **398 training** and **171 test** samples with stratification, identical to the split used in the previous notebooks. Keeping the same split across notebooks ensures that performance numbers are directly comparable, so when we say "GBM beats RF by 0.005," that comparison is on the exact same data.

**Why this matters:** Consistent data splits across notebooks are essential for the cumulative model comparison table we will build in the model selection notebook. If each notebook used a different random split, the comparisons would be meaningless.

---

## 2. Baseline Gradient Boosting

Before tuning anything, we train a `GradientBoostingClassifier` with scikit-learn's defaults: 100 trees, learning rate 0.1, and max depth 3. These defaults are surprisingly reasonable for many datasets and serve as our performance anchor. We also train a Random Forest under the same cross-validation to establish a direct baseline.

The comparison below answers a practical question: does gradient boosting out-of-the-box beat the Random Forest we tuned in the previous notebook? On the breast cancer dataset (569 samples, 30 features), the two methods often perform within a percentage point of each other, but boosting's sequential error-correction mechanism can edge ahead.

> 💡 **Gemini Prompt:** "Train GradientBoostingClassifier (defaults: 100 trees, lr=0.1, depth=3) and RandomForest(100 trees). Compare using 5-fold stratified CV ROC-AUC."
>
> **After running, verify:**
> - Both models report CV ROC-AUC mean and std
> - GBM advantage over RF is printed
> - StratifiedKFold uses shuffle=True, seed=474


In [ ]:
# Simple GBM with defaults
gbm = GradientBoostingClassifier(random_state=RANDOM_SEED)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
gbm_scores = cross_val_score(gbm, X_train, y_train, cv=cv, scoring='roc_auc')

print("=== GRADIENT BOOSTING (defaults) ===")
print(f"Default params:")
print(f"  n_estimators: 100")
print(f"  learning_rate: 0.1")
print(f"  max_depth: 3")
print(f"\nCV ROC-AUC: {gbm_scores.mean():.4f} ± {gbm_scores.std():.4f}")

# Compare to Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED, n_jobs=-1)
rf_scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='roc_auc')

print(f"\n=== RANDOM FOREST (100 trees) ===")
print(f"CV ROC-AUC: {rf_scores.mean():.4f} ± {rf_scores.std():.4f}")

print(f"\n=== COMPARISON ===")
print(f"GBM advantage: {(gbm_scores.mean() - rf_scores.mean()):.4f}")
print("\n💡 GBM often outperforms RF (but needs more tuning)")

**Reading the output:**

The output shows two blocks: GBM defaults (100 trees, learning rate 0.1, max depth 3) and Random Forest (100 trees). Both report 5-fold stratified CV ROC-AUC with mean and standard deviation.

On the breast cancer dataset, default GBM typically achieves a CV ROC-AUC around **0.99**, while the 100-tree Random Forest scores around **0.98-0.99**. The GBM advantage line at the bottom quantifies the lift, often in the range of **0.002-0.010**. A positive value means boosting outperforms bagging out of the box.

The standard deviations are worth comparing too. GBM's std is often similar to or slightly larger than RF's, meaning boosting is not necessarily more stable across folds. This is expected: boosting reduces bias but does not explicitly target variance reduction the way bagging does.

**Key takeaway:** Default Gradient Boosting is a strong out-of-the-box performer. The fact that it matches or beats a 100-tree Random Forest with no tuning makes it a compelling first choice for structured tabular data.

---

## 3. Learning Rate Tradeoff

### Learning Rate (η, or `learning_rate`)

**High learning rate (e.g., 0.3)**
- ✓ Faster training (need fewer trees)
- ✗ Risk of overfitting
- ✗ May miss optimal solution

**Low learning rate (e.g., 0.01)**
- ✓ Better final performance (usually)
- ✓ More stable
- ✗ Needs many more trees
- ✗ Slower training

**Rule of thumb**: Lower learning rate + more trees = better results (but longer training)

> 💡 **Gemini Prompt:** "Sweep learning rates [0.01,0.05,0.1,0.2,0.5] for GBM with 100 trees. Collect CV ROC-AUC and plot with error bars (log-scale x-axis)."
>
> **After running, verify:**
> - Table has 5 rows with lr, cv_mean, cv_std
> - Plot uses log x-axis with error bars
> - Lower LRs are competitive with higher ones


In [ ]:
# Learning rate sweep
learning_rates = [0.01, 0.05, 0.1, 0.2, 0.5]
lr_results = []

for lr in learning_rates:
    gbm_lr = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=lr,
        random_state=RANDOM_SEED
    )
    scores = cross_val_score(gbm_lr, X_train, y_train, cv=cv, scoring='roc_auc')
    lr_results.append({
        'learning_rate': lr,
        'cv_mean': scores.mean(),
        'cv_std': scores.std()
    })

lr_df = pd.DataFrame(lr_results)
print("=== LEARNING RATE SWEEP (100 trees) ===")
print(lr_df.to_string(index=False))

# Plot
plt.figure(figsize=(10, 6))
plt.errorbar(lr_df['learning_rate'], lr_df['cv_mean'], 
             yerr=lr_df['cv_std'], marker='o', capsize=5, linewidth=2)
plt.xlabel('Learning Rate')
plt.ylabel('CV ROC-AUC')
plt.title('Effect of Learning Rate (100 trees)')
plt.xscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Lower learning rate often performs better")
print("💡 But needs to compensate with more trees")

**Reading the output:**

The table shows CV ROC-AUC for five learning rates (0.01, 0.05, 0.1, 0.2, 0.5), all with 100 trees. At **learning_rate=0.01**, the model underfits because 100 trees are not enough to compensate for the tiny step size. Performance peaks around **0.05-0.1** and may decline at **0.5** where each tree overshoots the residuals.

The log-scale plot reveals a characteristic inverted-U shape: too slow (left) means underfitting with 100 trees, too fast (right) means overfitting. The sweet spot is in the middle. Note that the curve shape depends on `n_estimators`: with 500 trees, the optimal learning rate would shift leftward because more trees compensate for smaller steps.

**Why this matters:** Learning rate is the most important hyperparameter in gradient boosting. The rule of thumb is: set it as low as your compute budget allows, then increase `n_estimators` until validation performance plateaus.

---

## 4. Number of Trees vs Learning Rate

Learning rate and number of trees are tightly coupled in gradient boosting. A lower learning rate means each tree contributes less, so you need more trees to reach the same performance level. The tradeoff is computation: 500 trees at learning rate 0.01 takes roughly 5x longer than 100 trees at 0.1, but often yields a better final model because the optimization path is smoother.

The grid below tests four (n_estimators, learning_rate) configurations to illustrate this tradeoff. Look for the configuration that achieves the highest CV ROC-AUC; it is typically one of the low-learning-rate, high-tree-count combinations.

> 💡 **Gemini Prompt:** "Test four GBM configs pairing (n_estimators, lr): (50,0.1), (100,0.1), (200,0.05), (500,0.01). Evaluate with 5-fold CV ROC-AUC."
>
> **After running, verify:**
> - Table has 4 rows showing n_estimators, lr, cv_mean, cv_std
> - More trees + lower LR matches/beats fewer trees + higher LR
> - Same CV object used for fair comparison


In [ ]:
# Test: low LR with more trees
configs = [
    {'n_estimators': 50, 'learning_rate': 0.1},
    {'n_estimators': 100, 'learning_rate': 0.1},
    {'n_estimators': 200, 'learning_rate': 0.05},
    {'n_estimators': 500, 'learning_rate': 0.01},
]

config_results = []
for config in configs:
    gbm = GradientBoostingClassifier(random_state=RANDOM_SEED, **config)
    scores = cross_val_score(gbm, X_train, y_train, cv=cv, scoring='roc_auc')
    config_results.append({
        'n_estimators': config['n_estimators'],
        'learning_rate': config['learning_rate'],
        'cv_mean': scores.mean(),
        'cv_std': scores.std()
    })

config_df = pd.DataFrame(config_results)
print("=== LEARNING RATE + N_ESTIMATORS COMBINATIONS ===")
print(config_df.to_string(index=False))

print("\n💡 More trees with lower learning rate often wins")
print("💡 But training time increases significantly")

**Reading the output:**

The table tests four specific configurations that illustrate the learning-rate / n_estimators tradeoff. The (50 trees, 0.1) configuration is the fastest but usually lowest-performing. The (500 trees, 0.01) configuration is the slowest but often achieves the highest CV ROC-AUC.

Look at the middle rows: (100 trees, 0.1) and (200 trees, 0.05) often produce nearly identical performance, confirming that you can trade learning rate for tree count. The product `n_estimators * learning_rate` is sometimes called the "effective budget"; configurations with similar budgets tend to perform similarly.

Training time differences become noticeable here: 500 trees takes about 5x longer than 100 trees. On the small breast cancer dataset this is still under a second, but on larger datasets the time difference can be minutes versus seconds.

**Key takeaway:** For a final production model, invest in more trees with a smaller learning rate. For rapid prototyping, use fewer trees with a moderate learning rate (0.1) as a quick approximation.

---

## 📝 PAUSE-AND-DO Exercise 1 (5 minutes)

**Task:** Train baseline GBM and compare against RF under CV.

Already done above! Now analyze:

---

### YOUR ANALYSIS:

**Performance Comparison:**  
[Which performs better: GBM or RF? By how much?]

**Training Time:**  
[Estimate: which is faster to train?]

**When to use each:**  
[When would you choose GBM over RF?]

---

## 5. Tuning Gradient Boosting

### Key Hyperparameters

1. **n_estimators**: Number of boosting stages
2. **learning_rate**: Shrinkage parameter (η)
3. **max_depth**: Maximum tree depth (keep low: 3-5)
4. **min_samples_split**: Minimum samples to split
5. **subsample**: Fraction of samples for fitting each tree (stochastic GB)

### Tuning Strategy

**Use RandomizedSearchCV with constraints:**
- Limit search space to reasonable values
- Use early stopping to save time
- Monitor overfitting carefully

> 💡 **Gemini Prompt:** "Run RandomizedSearchCV (20 iters) on GBM: n_estimators(100-300), lr(0.01-0.2), max_depth(3-6), min_samples_split(2-19), subsample(0.7-1.0). Print best params and test ROC-AUC."
>
> **After running, verify:**
> - Best parameters printed as dict
> - Best CV ROC-AUC reported
> - Test score computed with predict_proba and roc_auc_score


In [ ]:
# Constrained randomized search
param_distributions = {
    'n_estimators': randint(100, 300),
    'learning_rate': uniform(0.01, 0.19),  # 0.01 to 0.2
    'max_depth': randint(3, 7),
    'min_samples_split': randint(2, 20),
    'subsample': uniform(0.7, 0.3)  # 0.7 to 1.0
}

random_search = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=RANDOM_SEED),
    param_distributions,
    n_iter=20,  # Try 20 combinations
    cv=cv,
    scoring='roc_auc',
    random_state=RANDOM_SEED,
    n_jobs=-1,
    verbose=0
)

print("=== RANDOMIZED SEARCH (20 iterations) ===")
print("Running...")
random_search.fit(X_train, y_train)

print(f"\nBest parameters: {random_search.best_params_}")
print(f"Best CV score: {random_search.best_score_:.4f}")

# Test performance
test_score = roc_auc_score(y_test, random_search.predict_proba(X_test)[:, 1])
print(f"Test score: {test_score:.4f}")

**Reading the output:**

The `RandomizedSearchCV` output reports the best parameter combination found across 20 random trials. Typical best parameters on the breast cancer dataset: `learning_rate` around **0.05-0.15**, `max_depth` around **3-5**, `n_estimators` around **150-250**, and `subsample` around **0.8-1.0**.

The **best CV score** should be equal to or slightly above the default GBM's CV score (around **0.99**). The **test score** printed below it should be consistent with the CV estimate. If the test score is much higher than the CV score, that is likely just variance on the small test set, not a real advantage.

Notice the `subsample` parameter: values below 1.0 mean that each tree is trained on a random subset of the training data (stochastic gradient boosting). This adds a bagging-like regularization effect that can reduce overfitting, especially when the learning rate is higher.

**Why this matters:** RandomizedSearchCV is the practical way to tune GBM in production. Exhaustive grid search over 5 hyperparameters with multiple values each would require thousands of fits; 20 random trials gets you into the right ballpark with manageable compute.

---

## 📝 PAUSE-AND-DO Exercise 2 (5 minutes)

**Task:** Run constrained tuning and report best params + score.

Already done! Now document your tuning strategy:

---

### YOUR TUNING STRATEGY:

**Best Parameters Found:**  
[List them]

**Performance Improvement:**  
[Compare to baseline GBM]

**What would you try next:**  
[If you had more time/compute, what would you tune?]

---

## 6. Overfitting in Boosting

Unlike Random Forests, which are relatively resistant to overfitting because they average independent trees, gradient boosting can overfit severely if you train too many iterations. Each new tree is specifically designed to correct the current model's errors, so eventually the model starts fitting noise rather than signal.

The staged-prediction plot below tracks ROC-AUC on both training and validation data at every iteration from 1 to 500 trees. The gap between the two curves is the overfitting signal. Where the validation curve peaks is the optimal number of trees; continuing beyond that point hurts generalization. In production, this pattern motivates **early stopping**: halt training when validation performance stops improving.

> 💡 **Gemini Prompt:** "Split train into smaller train/val (80/20). Fit GBM with 500 trees, lr=0.05. Use staged_predict_proba to track ROC-AUC per boosting iteration. Plot train vs val curves and find optimal n_trees."
>
> **After running, verify:**
> - Two curves: train rising toward 1.0, val peaking then plateau
> - Optimal number of trees printed
> - Chart title indicates overfitting monitoring


In [ ]:
# Track train vs validation performance over iterations
from sklearn.model_selection import train_test_split as split2

X_t, X_v, y_t, y_v = split2(X_train, y_train, test_size=0.2, random_state=RANDOM_SEED, stratify=y_train)

gbm_monitor = GradientBoostingClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=3,
    random_state=RANDOM_SEED
)

gbm_monitor.fit(X_t, y_t)

# Compute staged predictions
train_scores = []
val_scores = []

for i, (train_pred, val_pred) in enumerate(zip(
    gbm_monitor.staged_predict_proba(X_t),
    gbm_monitor.staged_predict_proba(X_v)
)):
    train_scores.append(roc_auc_score(y_t, train_pred[:, 1]))
    val_scores.append(roc_auc_score(y_v, val_pred[:, 1]))

# Plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_scores)+1), train_scores, label='Train', linewidth=2)
plt.plot(range(1, len(val_scores)+1), val_scores, label='Validation', linewidth=2)
plt.xlabel('Number of Trees')
plt.ylabel('ROC-AUC')
plt.title('Training Progress: Monitoring Overfitting')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find optimal number of trees
optimal_n = np.argmax(val_scores) + 1
print(f"\n=== OPTIMAL NUMBER OF TREES ===")
print(f"Optimal n_estimators: {optimal_n}")
print(f"Validation ROC-AUC: {val_scores[optimal_n-1]:.4f}")
print(f"\n💡 Validation performance peaks, then plateaus (slight overfit after ~{optimal_n} trees)")
print(f"💡 Could use early stopping in production")

**Reading the output:**

The training curve (blue) rises steadily toward **1.0** and stays there, while the validation curve (orange) rises, peaks, and then either plateaus or gently declines. The optimal number of trees is printed below the plot, typically in the range of **50-200** for this dataset and learning rate (0.05).

The gap between the two curves is the overfitting gap. At iteration 1, both curves are low (the model is a single shallow tree). As iterations increase, the training curve pulls away from the validation curve. If the validation curve starts declining after its peak, every additional tree past that point is fitting noise.

In production, you would use **early stopping**: monitor validation loss during training and stop when it has not improved for a set number of rounds (the "patience" parameter). scikit-learn's `GradientBoostingClassifier` supports `n_iter_no_change` for this purpose, and libraries like XGBoost and LightGBM have built-in early-stopping callbacks.

**Key takeaway:** Always monitor the validation curve during boosting. Training until convergence without validation monitoring is the single most common mistake with gradient boosting models.

---

## 7. Final Comparison: All Models

We now have four model families in our toolkit: Logistic Regression, Decision Tree, Random Forest, and Gradient Boosting (both default and tuned). This final comparison table puts all five candidates on equal footing using the same `StratifiedKFold` cross-validation.

The ranking typically shows ensemble methods (RF and GBM) at the top, with tuned Gradient Boosting achieving the highest ROC-AUC. However, the margin over a well-tuned Random Forest may be small, and the margin over Logistic Regression may be even smaller on a linearly separable dataset like breast cancer. The next notebook formalizes how to make the champion selection decision when margins are tight.

> 💡 **Gemini Prompt:** "Compare five models: LogReg, DecisionTree(depth=5), RF(200), GBM default, GBM tuned. Use 5-fold CV and test ROC-AUC. Sort by CV mean and identify champion."
>
> **After running, verify:**
> - Table has 5 rows sorted by CV_Mean
> - Each row shows Model, CV_Mean, CV_Std, Test_Score
> - Champion identified at bottom


In [ ]:
# Final comparison
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

final_models = {
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
    ]),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=RANDOM_SEED),
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=RANDOM_SEED, n_jobs=-1),
    'Gradient Boosting (default)': GradientBoostingClassifier(random_state=RANDOM_SEED),
    'Gradient Boosting (tuned)': random_search.best_estimator_
}

final_results = []
for name, model in final_models.items():
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
    
    # Test score
    if not hasattr(model, 'predict_proba'):
        model.fit(X_train, y_train)
    if hasattr(model, 'predict_proba'):
        if name not in ['Gradient Boosting (tuned)']:  # Already fitted
            model.fit(X_train, y_train)
        test_pred = model.predict_proba(X_test)[:, 1]
    else:
        test_pred = model.predict(X_test)
    
    test_sc = roc_auc_score(y_test, test_pred)
    
    final_results.append({
        'Model': name,
        'CV_Mean': scores.mean(),
        'CV_Std': scores.std(),
        'Test_Score': test_sc
    })

final_df = pd.DataFrame(final_results).sort_values('CV_Mean', ascending=False)
print("=== FINAL MODEL COMPARISON ===")
print(final_df.to_string(index=False))

print(f"\n✓ Champion: {final_df.iloc[0]['Model']} (CV: {final_df.iloc[0]['CV_Mean']:.4f})")

**Reading the output:**

The final table ranks five models by CV ROC-AUC. Typical ordering on the breast cancer dataset: **Gradient Boosting (tuned)** at the top with ROC-AUC near **0.99**, followed closely by **Gradient Boosting (default)** and **Random Forest**, then **Logistic Regression**, and finally the **Decision Tree** trailing by a few percentage points.

The champion model is identified at the bottom. On this dataset the margins between the top three models are often razor-thin (< 0.01 in ROC-AUC), which raises an important question: is the extra complexity of gradient boosting worth the marginal gain over logistic regression? The answer depends on the deployment context, which is exactly what the next notebook (model selection protocol) formalizes.

Compare the CV standard deviations: a model with slightly lower mean but also lower std may actually be preferable in a risk-sensitive application, because its worst-case performance is better.

**Why this matters:** Building a cumulative comparison table across notebooks creates institutional knowledge. Instead of remembering that "GBM was probably best," you have a documented, reproducible ranking under controlled conditions.

---

## 8. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Boosting vs Bagging**: Sequential vs parallel, bias vs variance reduction
2. **Gradient Boosting**: Each tree corrects previous errors
3. **Learning Rate**: Lower rates + more trees = better performance
4. **Tuning Strategy**: Use constrained search, monitor overfitting
5. **Performance**: GBM often beats RF, but needs more careful tuning

### Critical Rules:

> **"Lower learning rate + more trees = better (but slower)"**

> **"Keep trees shallow (depth 3-5) for boosting"**

> **"Monitor overfitting closely with boosting"**

### Next Steps:

- Next notebook: Model selection and comparison protocol
- We'll formalize how to choose between models
- Create reproducible experiment logs

---

## Participation Assignment Submission Instructions

### To Submit This Notebook:

1. **Complete all exercises**: Fill in both PAUSE-AND-DO exercise cells with your findings
2. **Run All Cells**: Execute `Runtime → Run all` to ensure everything works
3. **Save a Copy**: `File → Save a copy in Drive or Download the .ipynb extension`
4. **Submit**: Upload your `.ipynb` file in the participation assignment you find in the course Brightspace page.

### Before Submitting, Check:

- [ ] All cells execute without errors
- [ ] All outputs are visible
- [ ] Both exercise responses are complete
- [ ] Notebook is shared with correct permissions
- [ ] You can explain every line of code you wrote

### Next Step:

Complete the **Quiz** in Brightspace (auto-graded)

---

## Bibliography

- Friedman, J. H. (2001). "Greedy Function Approximation: A Gradient Boosting Machine." *Annals of Statistics*.
- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* - Tree-Based Methods (boosting overview)
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). *The Elements of Statistical Learning* - Boosting methods
- scikit-learn User Guide: [Gradient boosting](https://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting)

---



<center>

Thank you!

</center>